In [3]:
import sys
import yaml
with open('..//secrets.yml', 'r') as file:
    secrets = yaml.safe_load(file)
sys.path.append(secrets['elo_proj_path'])

In [4]:
from player_club_classes import team_elo, Player, Club, Match
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly
import pickle

from team_colors import team_color_dict
team_colors = pd.DataFrame(team_color_dict).T
team_colors.columns = ['Primary', 'Secondary']
team_colors = team_colors.rename_axis('Team').reset_index()

from plotly.validators.scatter.marker import SymbolValidator
raw_symbols = SymbolValidator().values

simple_symbols = [i for i in raw_symbols if str(i).isalpha()]

with open('../../Rugby_ELO/processed_data/playerbase.pickle', 'rb') as handle:
    playerbase = pickle.load(handle)
with open('../../Rugby_ELO/processed_data/matchlist.pickle', 'rb') as handle:
    matchlist = pickle.load(handle)
with open('../../Rugby_ELO/processed_data/teamlist.pickle', 'rb') as handle:
    teamlist = pickle.load(handle)

In [5]:
player_elo_list = []
for player_name, player in playerbase.items():
    player_elo = pd.DataFrame(player.elo_list, columns = [
        'Number', 'Full_Name', 'Team', 'Player', 'Position', 'Tries',
        'Try Assists', 'Conversion Goals', 'Penalty Goals',
        'Drop Goals Converted', 'Points', 'Passes', 'Runs', 'Meters Run',
        'Clean Breaks', 'Defenders Beaten', 'Offload', 'Turnovers Conceded',
        'Tackles', 'Missed Tackles', 'Lineouts Won', 'Penalties Conceded',
        'Yellow Cards', 'Red Cards', 'espn_id_num', 'Competition', 'Date',
        'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Minutes',
        'Position_Number', 'gameid', 'Unicode_ID', 'start_elo', 'end_elo'
       ])
    player_elo['Full Name'] = player_name[0]
    player_elo['Unicode_ID'] = player_name[1]
    player_elo_list.append(player_elo)

player_elo = pd.concat(player_elo_list).reset_index(drop=True)
player_elo = pd.merge(player_elo, team_colors, on = 'Team', how = 'left')
player_elo['elo_change'] = player_elo.end_elo - player_elo.start_elo
player_elo.Date = pd.to_datetime(player_elo.Date)

player_elo['week_num'] = player_elo['Date'].dt.isocalendar().week
player_elo['month'] = player_elo['Date'].dt.month
player_elo['year'] = player_elo['Date'].dt.year

In [6]:
def percentile(group):
    sz = group.size-1
    ranks = group.rank(method='max')
    return np.floor(100.0*(ranks-1)/sz)

starters = player_elo[player_elo.Position != 'R']
starters = starters.dropna(subset=['Position'])
current_players = starters[starters.groupby(['Full Name'])['Date'].transform(max) == starters['Date']]
current_players['percentile'] = np.floor(current_players.groupby('Position')['end_elo'].apply(percentile))
current_players = current_players[['Full_Name', 'Unicode_ID', 'percentile']]
current_players

C:\Users\Graeham\AppData\Local\Temp\ipykernel_20984\1572876672.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_players['percentile'] = np.floor(current_players.groupby('Position')['end_elo'].apply(percentile))


,Full_Name,Unicode_ID,percentile
264,Nick Abendanon,55485151,97.0
401,Mike Tindall,49485155,81.0
550,Anthony Allen,55545151,82.0
568,James Bailey,53485349,38.0
800,Ryan Lamb,55515350,48.0
...,...,...,...
504376,Ignacio Peculo,1101119510111511211095105100327310311097991051...,5.0
504377,Ignacio Dotti,1101119510111511211095105100327310311097991051...,7.0
504380,Jordi Jorba,1101119510111511211095105100327411111410010532...,14.0
504381,Alejandro Alonso Munoz,1101119510111511211095105100326510810110697110...,14.0


In [7]:
match_list = []
for _, match in matchlist.items():
    match_list.append({key:val for key, val in vars(match).items()})

In [16]:
import datetime

recent_games = [x for x in match_list if datetime.datetime.now() > x['date'] > datetime.datetime.now() - datetime.timedelta(days=7)]
[print(f'{x["away_team_name"]} at {x["home_team_name"]}') for x in recent_games]


France at Japan
Wales at South Africa
Scotland at Argentina
Argentina at Portugal
Ireland at New Zealand
England at Australia
Romania at Uruguay
Spain at Canada
Italy at Georgia


[None, None, None, None, None, None, None, None, None]

In [45]:
recent_game['home_elo']

1675.7560946117012

In [37]:
recent_game = recent_games[4]
home_team = pd.DataFrame(recent_game['home_team'][:, [0,1,31,-2, -1]], columns = ['Number', 'Full_Name', 'Minutes', 'Unicode_ID', 'elo'])
home_team = home_team.merge(current_players, on=['Full_Name', 'Unicode_ID'])
home_team = home_team.drop(['Unicode_ID'], axis = 1)
away_team = pd.DataFrame(recent_game['away_team'][:, [0,1,31,-2, -1]], columns = ['Number', 'Full_Name', 'Minutes', 'Unicode_ID', 'elo'])
away_team = away_team.merge(current_players, on=['Full_Name', 'Unicode_ID'])
away_team = away_team.drop(['Unicode_ID'], axis = 1)

home_team.columns = ['Number', 'Home Player', 'Home Minutes', 'Home elo', 'Home Percentile']
away_team.columns = ['Number', 'Away Player', 'Away Minutes', 'Away elo', 'Away Percentile']

all_players = pd.merge(home_team, away_team)
all_players = all_players.sort_values('Number')
all_players = all_players[['Away Minutes', 'Away Player', 'Away elo','Away Percentile', 'Number', 'Home Percentile', 'Home elo', 'Home Player', 'Home Minutes']]
all_players

,Away Minutes,Away Player,Away elo,Away Percentile,Number,Home Percentile,Home elo,Home Player,Home Minutes
7,65.0,Andrew Porter,105.791011,89.0,1.0,23.0,93.745562,George Bower,64.0
2,64.0,Dan Sheehan,81.261693,13.0,2.0,66.0,101.597315,Codie Taylor,57.0
4,65.0,Tadhg Furlong,103.024992,84.0,3.0,94.0,113.700592,Ofa Tu'ungafasi,81.0
3,81.0,Tadhg Beirne,108.25333,90.0,4.0,98.0,126.019121,Brodie Retallick,69.0
6,69.0,James Ryan,100.072725,73.0,5.0,95.0,117.793807,Scott Barrett,74.0
8,76.0,Peter O'Mahony,105.402545,89.0,6.0,89.0,108.457896,Dalton Papalii,59.0
15,81.0,Josh van der Flier,120.38603,98.0,7.0,99.0,125.56057,Sam Cane,81.0
5,69.0,Caelan Doris,96.760505,55.0,8.0,87.0,111.091316,Ardie Savea,33.0
1,69.0,Jamison Gibson-Park,112.194505,95.0,9.0,94.0,113.885827,Aaron Smith,62.0
14,74.0,Johnny Sexton,128.806876,99.0,10.0,98.0,124.983873,Beauden Barrett,57.0


In [6]:
future_games = [x for x in match_list if 'point_diff' not in x.keys()]

In [36]:
future_game = future_games[0]

home_team = pd.DataFrame(future_game['home_team'][:, [0,1,-2, -1]], columns = ['Number', 'Full_Name', 'Unicode_ID', 'elo'])
home_team = home_team.merge(current_players, on=['Full_Name', 'Unicode_ID'])
home_team = home_team.drop(['Unicode_ID'], axis = 1)
home_team = home_team.sort_values('Number')

away_team = pd.DataFrame(future_game['away_team'][:, [0,1,-2, -1]], columns = ['Number', 'Full_Name', 'Unicode_ID', 'elo'])
away_team = away_team.merge(current_players, on=['Full_Name', 'Unicode_ID'])
away_team = away_team.drop(['Unicode_ID'], axis = 1)
away_team = away_team.sort_values('Number')

home_team.columns = ['Number', 'Home Name', 'Home elo', 'Home Percentile']
away_team.columns = ['Number', 'Away Name', 'Away elo', 'Away Percentile']

all_players = pd.merge(home_team, away_team)
all_players = all_players.sort_values('Number')
all_players

NameError: name 'future_games' is not defined

In [40]:
## Draft attribution chart
def lineup_contribition(player_df, column):
    mult = np.where(player_df.Number <= 15, (7/8), 0.234)
    return player_df[column] * mult

all_players['home contributions'] = lineup_contribition(all_players, 'Home elo')
all_players['away contributions'] = lineup_contribition(all_players, 'Away elo')
all_players['home minute_elos'] = all_players['Home elo'] * all_players['Home Minutes'] / max(all_players['Home Minutes'])
all_players['away minute_elos'] = all_players['Away elo'] * all_players['Away Minutes'] / max(all_players['Home Minutes'])

all_players[['Number', 'home contributions', 'away contributions', 'home minute_elos', 'away minute_elos']]

,Number,home contributions,away contributions,home minute_elos,away minute_elos
7,1.0,82.027367,92.567135,74.070568,84.894021
2,2.0,88.897651,71.103982,71.494407,64.20677
4,3.0,99.488018,90.146868,113.700592,82.674376
3,4.0,110.266731,94.721663,107.349621,108.25333
6,5.0,103.069581,87.563634,107.614096,85.247136
8,6.0,94.900659,92.227227,79.000196,98.896215
15,7.0,109.865499,105.337776,125.56057,120.38603
5,8.0,97.204902,84.665442,45.259425,82.425616
1,9.0,99.650098,98.170192,87.171867,95.573097
14,10.0,109.360889,112.706017,87.951615,117.675418


In [41]:
long_players = all_players[['Number', 'home contributions', 'away contributions', 'home minute_elos', 'away minute_elos']].melt(id_vars = 'Number')
fig = px.bar(long_players, x="variable", y="value", color = 'Number')
fig.show()

In [42]:
import plotly.graph_objects as go

pos_plotlist = []
for _, row in all_players.iterrows():
    pos_plot = go.Bar(
        name=row.Number,
        x = ['Home Projected', 'Home Actual', 'Away Projected', 'Away Actual'],
        y = [row['home contributions'], row['home minute_elos'], row['away contributions'], row['away minute_elos']],
        customdata = [row['Home Player'], row['Home Player'], row['Away Player'], row['Away Player']],
        hovertemplate = 
            "<b>%{customdata}</b>: " + 
            "%{y}"
    )
    pos_plotlist.append(pos_plot)

fig = go.Figure(data=pos_plotlist)
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [43]:
fig = go.Figure(data=pos_plotlist)
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [41]:
row = all_players.iloc[0]
row

Number                               1.0
Home Name                  Keita Inagaki
Home elo                       95.693948
Home Percentile                     34.0
Away Name             Jean-Baptiste Gros
Away elo                      114.835326
Away Percentile                     96.0
home contributions             83.732204
away contributions             100.48091
Name: 0, dtype: object

In [9]:
month = match_df[(match_df.year == 2022) & (match_df.month == 7)]
closest_games = month.sort_values('lineup_spread',  key=abs).head()

sorted_pred_accuracy = month.assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs)#.drop(columns="tmp")
best_preds = sorted_pred_accuracy.head()
worst_preds = sorted_pred_accuracy.tail()

biggest_upsets = month[(month.lineup_spread * month.point_diff) < 0].assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs, ascending=False).drop(columns="tmp").head()
biggest_covers = month[((month.lineup_spread * month.point_diff) > 0)].assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs, ascending=False)#.drop(columns="tmp").head()

NameError: name 'match_df' is not defined

In [7]:
# CHANGE
player_month = player_elo[(player_elo.year == 2022) & (player_elo.month == 5)]
oneplayer = player_month[(player_month.Unicode_ID == '5155544848') & (player_month['Full Name'] == 'Caleb Delany')].sort_values('Date')
oneplayer.iloc[-1].end_elo - oneplayer.iloc[0].start_elo

-2.554117441069806

In [11]:
def elo_change(df):
    return df.iloc[-1].end_elo, df.iloc[0].start_elo, df.iloc[-1].end_elo - df.iloc[0].start_elo

elo_changes = player_month.sort_values('Date').groupby(['Unicode_ID', 'Full Name', 'Team']).apply(elo_change).reset_index()
elo_changes[['end_elo', 'start_elo', 'elo_change']] = pd.DataFrame(elo_changes[0].tolist(), index=elo_changes.index)
elo_changes.sort_values('elo_change')

,Unicode_ID,Full Name,Team,0,end_elo,start_elo,elo_change
576,5051515156,James Blackwell,Hurricanes,"(100.88391371265058, 106.42990302317858, -5.54...",100.883914,106.429903,-5.545989
826,5055485550,Salesi Rayasi,Hurricanes,"(110.70188334521939, 115.67822631062886, -4.97...",110.701883,115.678226,-4.976343
1070,5149525654,Quinn Tupaea,Chiefs,"(113.26707089934938, 118.08496318931884, -4.81...",113.267071,118.084963,-4.817892
267,5048485452,Ardie Savea,Hurricanes,"(118.33886733853684, 122.95562106235826, -4.61...",118.338867,122.955621,-4.616754
589,5051515748,Josh Ioane,Chiefs,"(98.22976447566921, 102.77928260508381, -4.549...",98.229764,102.779283,-4.549518
...,...,...,...,...,...,...,...
406,5049535052,Mattia Bellini,Benetton Treviso,"(96.79422627952788, 92.30762615721996, 4.48660...",96.794226,92.307626,4.486600
745,5053505254,Rhyno Smith,Benetton Treviso,"(99.09602275738854, 94.60942263508062, 4.48660...",99.096023,94.609423,4.486600
244,4957555455,Juan Ignacio Brex,Benetton Treviso,"(98.82188475913395, 94.33528463682603, 4.48660...",98.821885,94.335285,4.486600
408,5049535155,Federico Ruzza,Benetton Treviso,"(102.4315904715746, 97.94499034926667, 4.48660...",102.431590,97.944990,4.486600
